In [1]:
import cv2,os,sys
os.sys.path.append(os.getcwd()+'/src/')
import argparse
import logging
import time
import ast

import common
import numpy as np
from estimator import TfPoseEstimator
from networks import get_graph_path, model_wh
import multiprocessing
import time
import cv2
import sys 

import matplotlib.pyplot as plt

/home/anuj/.virtualenvs/video-analytics-2/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
video_path = '/home/anuj/Desktop/datasets/test-1.mp4'

In [ ]:
def human_detector_without_fps_logic(video_path, input_shape, channel=2, zoom=1, model_name='mobilenet_thin', increase_fps=False):
    
    channel = channel
    zoom = zoom
    im_w, im_h = input_shape
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(im_w, im_h))
    fps = 20
    
    cap = cv2.VideoCapture(video_path)
    
    n_row = 2
    while True:
        if increase_fps:
            for i in range(4):
                cap.read()
            
        fps_time = time.time()
        
        ret, image = cap.read()
         
        if zoom < 1.0:
            canvas = np.zeros_like(image)
            img_scaled = cv2.resize(image, None, fx=zoom, fy=zoom, interpolation=cv2.INTER_LINEAR)
            dx = (canvas.shape[1] - img_scaled.shape[1]) // 2
            dy = (canvas.shape[0] - img_scaled.shape[0]) // 2
            canvas[dy:dy + img_scaled.shape[0], dx:dx + img_scaled.shape[1]] = img_scaled
            image = canvas
        elif zoom > 1.0:
            img_scaled = cv2.resize(image, None, fx=zoom, fy=zoom, interpolation=cv2.INTER_LINEAR)
            dx = (img_scaled.shape[1] - image.shape[1]) // 2
            dy = (img_scaled.shape[0] - image.shape[0]) // 2
            image = img_scaled[dy:image.shape[0], dx:image.shape[1]]

        # detect human using tf-pose
        humans = e.inference(image)

        # modified tf-pose-code
        # get all points of a human 
        image, centers_humans = e.draw_humans(image, humans, imgcopy=False)

        # find the bounding rectangle out of the human points
        for centers in centers_humans.values():
            rect = cv2.boundingRect(np.array( centers.values()))
            (x,y,w,h) = rect
            image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255),2)

        cv2.putText(image,
                    "FPS: %f" % (1.0 / (time.time() - fps_time)),
                    (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (0, 255, 0), 2)
        cv2.namedWindow('tf_pose_estimation_result',cv2.WINDOW_NORMAL)
        cv2.imshow('tf_pose_estimation_result', image)
        fps_time = time.time()
        if cv2.waitKey(1) == 27:
            break

    cv2.destroyAllWindows()
    cap.release()

In [ ]:
human_detector_without_fps_logic(video_path, (640, 352), increase_fps=True)

# ---------------- Up above is working ------------------------

In [ ]:
def human_detector_with_fps_logic(q, video_path, input_shape, channel=2, zoom=1, model_name='mobilenet_thin', increase_fps=False):
    """
    model parameters: 
    """
    
    channel = channel
    zoom = zoom
    im_w, im_h = input_shape
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(im_w, im_h))
    fps = 20
    
    cap = cv2.VideoCapture(video_path)
    
    count = 1
    n_row = 2
    while True:
        if increase_fps:
            for i in range(4):
                cap.read()
            
        fps_time = time.time()
        
        # read n frames from 'VideoCapture' buffer
        concatenated_frame = []
        for row in range(n_row):
            frame_arr = []
            for i in range(n_row):
                ret, frame = cap.read()
                frame_arr.append(frame)

            concat_frame = np.concatenate(frame_arr,axis=1)
            concatenated_frame.append(concat_frame)


        image = np.concatenate(concatenated_frame,axis=0)
        
        if zoom < 1.0:
            canvas = np.zeros_like(image)
            img_scaled = cv2.resize(image, None, fx=zoom, fy=zoom, interpolation=cv2.INTER_LINEAR)
            dx = (canvas.shape[1] - img_scaled.shape[1]) // 2
            dy = (canvas.shape[0] - img_scaled.shape[0]) // 2
            canvas[dy:dy + img_scaled.shape[0], dx:dx + img_scaled.shape[1]] = img_scaled
            image = canvas
        elif zoom > 1.0:
            img_scaled = cv2.resize(image, None, fx=zoom, fy=zoom, interpolation=cv2.INTER_LINEAR)
            dx = (img_scaled.shape[1] - image.shape[1]) // 2
            dy = (img_scaled.shape[0] - image.shape[0]) // 2
            image = img_scaled[dy:image.shape[0], dx:image.shape[1]]

        # detect human using tf-pose
        humans = e.inference(image)

        # modified tf-pose-code
        # get all points of a human 
        image, centers_humans = e.draw_humans(image, humans, imgcopy=False)

        # find the bounding rectangle out of the human points
        for centers in centers_humans.values():
            rect = cv2.boundingRect(np.array( centers.values()))
            (x,y,w,h) = rect
            image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255),2)

            
        ######################### DE-STITCH LOGIC #######################
        x = 0
        y = 0
        destiched_frame_arr = []
        for i in range(n_row):
            for j in range(n_row):
                destiched_frame_arr.append(image[y:y+im_h, x:x+im_w])
                x += im_w
            y += im_h
            x = 0


        for i in range(len(destiched_frame_arr)):
            print('filling queue: ', count)
            
            q.put(frame)
#             cv2.imshow('final', destiched_frame_arr[i])
#             out.write(destiched_frame_arr[i])
        count += 1
        ######################### DE-STITCH LOGIC #######################
        

#         cv2.putText(image,
#                     "FPS: %f" % (1.0 / (time.time() - fps_time)),
#                     (10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,
#                     (0, 255, 0), 2)

#         cv2.namedWindow('tf_pose_estimation_result',cv2.WINDOW_NORMAL)
#         cv2.imshow('tf_pose_estimation_result', image)
        fps_time = time.time()
#         if cv2.waitKey(1) == 27:
#             break
    
    cv2.destroyAllWindows()
    cap.release()
    # out.release()    

In [ ]:
# human_detector_with_fps_logic(video_path, (640, 352), increase_fps=True)

In [ ]:
def fn2(q):
    while True:
        x = q.get()
        cv2.imshow('hello', x)
        k = cv2.waitKey(1)
        if k == 27:
            break

    q.close()
    cv2.destroyAllWindows()

In [ ]:
queue = multiprocessing.Queue()

proc1 = multiprocessing.Process(target=human_detector_with_fps_logic, args=(queue, video_path, (640, 352), False,))
proc2 = multiprocessing.Process(target=fn2, args=(queue,))
# proc1.daemon = True
# proc2.daemon = True
proc1.start()
proc2.start()




# proc1.terminate()
# proc2.terminate()

# print('CAME HERE')

queue.close()
queue.join_thread()


proc1.join()
proc2.join()

# ------------------------------ up above is working ------------------------

In [3]:
def check_intersection(a, b):
    """
    a : rectangle in (x,y,w,h)
    """
    x = max(a[0], b[0])
    y = max(a[1], b[1])
    w = min(a[0]+a[2], b[0]+b[2]) - x
    h = min(a[1]+a[3], b[1]+b[3]) - y
    if w<0 or h<0: 
        return False,() # or (0,0,0,0) ?
    return True,(x, y, w, h)

In [4]:
def intrusion_detector(video_path, input_shape, bbox=None, channel=2, zoom=1, model_name='mobilenet_thin', increase_fps=False):
    
    #-------------- fps increment settings ---
    inc_fps_flag, num_frames_to_skip = (increase_fps)
        
    if inc_fps_flag:
        if num_frames_to_skip < 1:
            print('entered frames to skip is less than 1, please input a higher value')
            sys.exit()
        if num_frames_to_skip > 7:
            print('entered frames to skip is more than 7, please input a value less than 7')
            sys.exit()
    #-------------- fps increment settings ---
    
    
    cap = cv2.VideoCapture(video_path)
    
    #--------------- intrusion parameters ----------
    bbox = bbox
    text = 'select minimum area object \n skip frame : press s\nconfirmation : press c'
    #--------------- intrusion parameters ----------
    
    
    # pose estimator parameters
    channel = channel
    zoom = zoom
    im_w, im_h = input_shape
    e = TfPoseEstimator(get_graph_path('mobilenet_thin'), target_size=(im_w, im_h))
    
    
    while cap.isOpened():
        
        if inc_fps_flag:
            for i in range(inc_fps_flag):
                cap.read()
        
        
        try:
            ret, image = cap.read()
            ####################### start  ########################
            

            #----------------------------------- ROI selector ---------------------------
            while bbox is None:
                cv2.namedWindow('select ROI', cv2.WINDOW_NORMAL)
                for idx,i in enumerate(text.split('\n')):
                    cv2.putText(image, i, (10,40*(idx+1)), cv2.FONT_HERSHEY_PLAIN, 1,(0,255,0),1,cv2.LINE_AA)


                k = cv2.waitKey(1)
                cv2.imshow('select ROI', image)
                if k==115:
                    ##  press s
                    bbox = None
                    ret, image = cap.read()
                    
                if k==99:
                    ## press c
                    bbox = cv2.selectROI('select ROI', image)
                    cv2.destroyWindow('select ROI')
                    if not any(bbox):
                        print("NO ROI SELECTED, Exiting!")
                        sys.exit(0)
                    print("Selected ROI Coordinates: " + str(bbox))
            #----------------------------------- ROI selector ---------------------------
            
                
            if zoom < 1.0:
                canvas = np.zeros_like(image)
                img_scaled = cv2.resize(image, None, fx=zoom, fy=zoom, interpolation=cv2.INTER_LINEAR)
                dx = (canvas.shape[1] - img_scaled.shape[1]) // 2
                dy = (canvas.shape[0] - img_scaled.shape[0]) // 2
                canvas[dy:dy + img_scaled.shape[0], dx:dx + img_scaled.shape[1]] = img_scaled
                image = canvas
            elif zoom > 1.0:
                img_scaled = cv2.resize(image, None, fx=zoom, fy=zoom, interpolation=cv2.INTER_LINEAR)
                dx = (img_scaled.shape[1] - image.shape[1]) // 2
                dy = (img_scaled.shape[0] - image.shape[0]) // 2
                image = img_scaled[dy:image.shape[0], dx:image.shape[1]]


            # detect human using tf-pose
            humans = e.inference(image)
            

            # modified tf-pose-code
            # get all points of a human 
            image, centers_humans = e.draw_humans(image, humans, imgcopy=False)
            
            rect_array = []
            # find the bounding rectangle out of the human points
            for centers in centers_humans.values():
                rect = cv2.boundingRect(np.array( centers.values()))
                rect_array.append(rect)
                (x,y,w,h) = rect
                image = cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,255),2)
            
            
            res = []
            for rect in rect_array :
                res.append(check_intersection(np.array(rect), np.array(bbox))[0])
            
            
            if any(res):
                num_humans = sum(res)
                print('Number of humans' + str(num_humans))
            else:
                print('no humans in roi')
            
            
            cv2.rectangle(image, (bbox[0],bbox[1]),(bbox[0]+bbox[2],bbox[1]+bbox[3]),(255,0,0),1)
            ####################### end  ########################
            cv2.imshow('intrusion', image)
            k = cv2.waitKey(1)
            if k == 27:
                break

        except Exception as e:
            print(e)
            break


    cap.release()
    cv2.destroyAllWindows()

In [5]:
intrusion_detector(video_path, input_shape=(640, 352), increase_fps=(True, 6))

Selected ROI Coordinates: (288, 124, 89, 183)
no humans in roi
no humans in roi
no humans in roi
no humans in roi
no humans in roi
no humans in roi
no humans in roi
no humans in roi
no humans in roi
no humans in roi
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of humans1
Number of 